In [1]:
import scanpy as sc
import random
from math import floor
import pathlib
from cmonge.utils import load_config
import yaml
import os

In [2]:
random.seed(11111)

In [3]:
adata = sc.read_h5ad("/Users/adr/Box/otperturb/datasets/scrna-sciplex3/hvg.h5ad")

In [4]:
all_drugs = adata.obs["drug"].unique().tolist()
all_drugs.remove("control")
all_drugs.remove("enmd_2076_l__tartaric_acid") # Not evaluated/used for some reason
all_conditions = [f"{d}-{dose}" for d in all_drugs for dose in ["10", "100", "1000", "10000"]]

print(floor(len(all_drugs)*0.05))

9


In [5]:
while len(all_drugs)>=9:
    ood_drugs = random.sample(all_drugs, 9)
    all_drugs = [d for d in all_drugs if not d in ood_drugs] # Sets don't work well with random sampling for reproducible results
    split_name = ood_drugs[0].split()[0]
    print(split_name, len(all_drugs))
    with open(f"/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/{split_name}_ood_conditions.txt", "w") as f:
        for d in ood_drugs:
            for dose in ["-10", "-100", "-1000", "-10000"]:
                f.write(f"{d}{dose}\n")
    with open(f"/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/{split_name}_ood_drugs.txt", "w") as f:
        for d in ood_drugs:
                f.write(f"{d}\n")

if len(all_drugs)>0:
    ood_drugs = all_drugs
    all_drugs = [d for d in all_drugs if not d in ood_drugs]
    split_name = f"SMALLER_SPLIT_{ood_drugs[0].split()[0]}"
    print(split_name, len(all_drugs))
    with open(f"/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/{split_name}_ood_conditions.txt", "w") as f:
        for d in ood_drugs:
            for dose in ["-10", "-100", "-1000", "-10000"]:
                f.write(f"{d}{dose}\n")
    with open(f"/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/{split_name}_ood_drugs.txt", "w") as f:
        for d in ood_drugs:
                f.write(f"{d}\n")

ruxolitinib 178
roscovitine 169
tazemetostat 160
filgotinib 151
nintedanib 142
quercetin 133
givinostat 124
azacitidine 115
sl_327 106
bisindolylmaleimide_ix 97
xav_939 88
iniparib 79
abt_737 70
mc1568 61
meprednisone 52
dasatinib 43
alvespimycin_hcl 34
sns_314 25
prednisone 16
itsa_1 7
SMALLER_SPLIT_eed226 0


In [6]:
all_drugs

[]

# Create configs for training CMonge on vela

In [7]:
cond_files = [str(f.absolute()) for f in pathlib.Path("/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/").glob("*_conditions.txt")]
cond_files

['/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/filgotinib_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/iniparib_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/meprednisone_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/mc1568_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/ruxolitinib_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/tazemetostat_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/itsa_1_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/alvespimycin_hcl_ood_conditions.txt',
 '/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/all_drugs/5perc_ood/nintedanib_ood_conditions.txt',
 '/Users/

In [8]:
base_moa_config = load_config("/Users/adr/Projects/enlight/enlight/OT_analysis/condmonge/vela/configs/cmonge_9_ood_big_moa.yaml")
base_rdkit_config = load_config("/Users/adr/Projects/enlight/enlight/OT_analysis/condmonge/vela/configs/cmonge_9_ood_big_RDkit.yaml")

In [10]:
for split in cond_files:
    with open(split, "r") as f:
        ood_conditions = f.readlines()
    ood_conditions = [c.strip() for c in ood_conditions]
    id_conditions = [c for c in all_conditions if not c in ood_conditions]

    ood_drug_file = f"{split[:-15]}_drugs.txt"
    with open(ood_drug_file, "r") as f:
        ood_drugs = f.readlines()
    ood_drugs = [d.strip() for d in ood_drugs]
    
    moa_config = load_config("/Users/adr/Projects/enlight/enlight/OT_analysis/condmonge/vela/configs/cmonge_9_ood_big_moa.yaml")
    rdkit_config = load_config("/Users/adr/Projects/enlight/enlight/OT_analysis/condmonge/vela/configs/cmonge_9_ood_big_RDkit.yaml")

    moa_config.condition.conditions = id_conditions
    moa_config.logger_path = f"/data/cmonge/models/5perc_ood/moa/{ood_drugs[0]}_ood/"
    moa_config.model.checkpointing_args.checkpoint_dir = f"/data/cmonge/models/5perc_ood/moa/{ood_drugs[0]}_ood/model/"
    moa_config.ood_condition.conditions = ood_conditions
    moa_config.ood_drug = "-".join(ood_drugs)
    # moa_config.data.batch_size = 2048
    # moa_config.model.optim.grad_acc_steps = 1

    rdkit_config.condition.conditions = id_conditions
    rdkit_config.logger_path = f"/data/cmonge/models/5perc_ood/rdkit/{ood_drugs[0]}_ood/"
    rdkit_config.model.checkpointing_args.checkpoint_dir = f"/data/cmonge/models/5perc_ood/rdkit/{ood_drugs[0]}_ood/model/"
    rdkit_config.ood_condition.conditions = ood_conditions
    rdkit_config.ood_drug = "-".join(ood_drugs)
    # rdkit_config.data.batch_size = 2048
    # rdkit_config.model.optim.grad_acc_steps = 1

    assert ood_drugs[0] in split
    
    with open(f"/Users/adr/Box/otperturb/alice_exp/sciplex_5perc_ood/configs/{ood_drugs[0]}_ood_moa_config.yaml", "w") as f:
        yaml.dump(moa_config.toDict(), f, default_flow_style=False)
    with open(f"/Users/adr/Box/otperturb/alice_exp/sciplex_5perc_ood/configs/{ood_drugs[0]}_ood_rdkit_config.yaml", "w") as f:
        yaml.dump(rdkit_config.toDict(), f, default_flow_style=False)

    try:
        os.makedirs(f"/Users/adr/Box/otperturb/alice_exp/sciplex_5perc_ood/moa/{ood_drugs[0]}_ood/")
    except FileExistsError:
        print("Directory exists")
    try:
        os.makedirs(f"/Users/adr/Box/otperturb/alice_exp/sciplex_5perc_ood/rdkit/{ood_drugs[0]}_ood/")
    except FileExistsError:
        print("Directory exists")
    
    with open(f"/Users/adr/Box/otperturb/alice_exp/sciplex_5perc_ood/moa/{ood_drugs[0]}_ood/config.yaml", "w") as f:
        yaml.dump(moa_config.toDict(), f, default_flow_style=False)
    with open(f"/Users/adr/Box/otperturb/alice_exp/sciplex_5perc_ood/rdkit/{ood_drugs[0]}_ood/config.yaml", "w") as f:
        yaml.dump(rdkit_config.toDict(), f, default_flow_style=False)
    

Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
Directory exists
